![lenet结构图](../../workspace/pic/lenet.jpg)
1. 卷积的输出形状由批量大小、通道数、高度、宽度决定
2. 每个2x2池化层将空间维数减少4倍
3. 3个全连接层，分别有120、84、10个输出

In [28]:
import torchvision
import torch
from torch.utils.data import DataLoader
from torch import nn
from torch.utils.tensorboard import SummaryWriter
# from lenet import *


class LeNet5(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=4, stride=1)  # 修改卷积核大小为4x4
        self.avgpool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=4, stride=1)  # 修改卷积核大小为4x4
        self.avgpool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 修改全连接层输入维度为16 * 1 * 1
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.avgpool1(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.avgpool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        return x


In [29]:
train_data = torchvision.datasets.FashionMNIST(root = "./workspace/data", train=True, transform=torchvision.transforms.ToTensor(),
                                         download = False)
test_data = torchvision.datasets.FashionMNIST(root = "./workspace/data", train=False, transform=torchvision.transforms.ToTensor(),
                                         download = False)

train_data_size = len(train_data)
test_data_size = len(test_data)

print(train_data_size)

train_dataloader = DataLoader(train_data,batch_size = 256)
test_dataloader = DataLoader(test_data,batch_size = 256)

60000


In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


lenet = LeNet5()
lenet = lenet.to(device)

loss_fn = nn.CrossEntropyLoss()
loss_fn.to(device)

learning_rate = 0.01
optimizer = torch.optim.SGD(lenet.parameters(),lr=learning_rate)

In [31]:
# 设置训练相关参数

total_train_step = 0
total_test_step = 0
epoch = 80


In [32]:
writer = SummaryWriter("lenet_train")

for i in range(epoch):
    print("----------------第{}轮训练开始----------------".format(i+1))
    
    # 训练
    for data in train_dataloader:
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        
        outputs = lenet(imgs)    #网络推理得到输出
        loss = loss_fn(outputs,targets)   #计算预测值与真实值之间的差值
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_train_step += 1
        if total_train_step % 100 == 0:
            print("训练次数:{}, Loss:{}".format(total_train_step, loss.item())) # 这里的loss.item()可以返回非tensor的正常数字，方便可视化
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    # 测试
    total_test_loss = 0
    total_accuracy = 0
    
    with torch.no_grad():      #这一步不参与梯度计算
        for data in test_dataloader:
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            
            outputs = lenet(imgs)
            loss = loss_fn(outputs,targets)
            total_test_loss = total_test_loss + loss.item()
            
            accuracy = (outputs.argmax(1) == targets).sum()   # argmax(1) 代表在每一行搜索最大值
            total_accuracy = total_accuracy + accuracy
            
    
    print("整体测试集上的Loss:{}".format(total_test_loss))
    print("整体测试集上的正确率：{}".format(total_accuracy / test_data_size))     
            
    writer.add_scalar("test_accuracy", total_accuracy / test_data_size, total_test_step)
    writer.add_scalar("test_loss", total_test_loss, total_test_step)

    total_test_step = total_test_step + 1
    
torch.save(lenet,"./workspace/model/lenet_80.pth")
    # torch.save(lenet.state_dict(),"lenet_{}.pth".format(i))
    
print("保存模型！！！")
    
writer.close()
        

----------------第1轮训练开始----------------


RuntimeError: mat1 and mat2 shapes cannot be multiplied (256x256 and 16x120)